<a href="https://colab.research.google.com/github/mightyssge/mightyssge/blob/main/Notebooks/ABB_MLlib_Transformacion_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img style="float: left; padding: 0px 10px 0px 0px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/Universidad_de_Lima_logo.png/220px-Universidad_de_Lima_logo.png"  width="120" />  MLlib: Transformaciones de Datos
**Profesor:** Enver G. Tarazona Vargas <br>
**Curso:** Analítica con Big Data <br>
**FACULTAD DE INGENIERÍA - CARRERA DE INGENIERÍA DE SISTEMAS**<br>

Usualmente no se tiene los datos en un formato conveniente. Una gran parte del trabajo con datos consiste en usar el conocimiento de un dominio determinado para saber cómo manejar los datos (eliminar algunos datos faltantes, realizar "feature engineering", transformar datos, etc.)

Spark tiene métodos para realizar estas transformaciones: http://spark.apache.org/docs/latest/ml-features.html

In [12]:
!pip install -q pyspark

In [13]:
c

NameError: name 'c' is not defined

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

## 1.&nbsp;Atributos categóricos a numéricos usando índices

Se puede utilizar `StringIndexer` para convertir atributos categóricos (no numéricos) en atributos numéricos.

In [ ]:
from pyspark.ml.feature import StringIndexer

*Ejemplo*: creación manual de un dataframe donde el atributo "sucursal" es categórico (no es numérico)

In [ ]:
from pyspark.sql import Row

df = spark.createDataFrame([Row(ID=0, sucursal="A", venta=10000), Row(ID=1, sucursal="B", venta=9000),
                            Row(ID=2, sucursal="C", venta=15000), Row(ID=3, sucursal="A", venta=14000),
                            Row(ID=4, sucursal="A", venta=12000), Row(ID=5, sucursal="C", venta=19000),
                            Row(ID=6, sucursal="D", venta=11500), Row(ID=7, sucursal="D", venta=5000)
])
df.show()

Se convertirá la columna "sucursal" en numérica utilizando `StringIndexer`. Primero se indica cuál es la columna de entrada (`inputCol`) y cuál será la columna de salida (`indiceSucursal`)

In [ ]:
# La columna con categoría indexada (numérica) se llamará "indiceCategoria"
indexador = StringIndexer(inputCol="sucursal", outputCol="indiceSucursal")

# Obtener las asociaciones entre categorías y valores numéricos (mapa)
modeloIndexador = indexador.fit(df)
# Mostrar las etiquetas que se mapean como (0, 1, 2)
modeloIndexador.labels

In [ ]:
modeloIndexador

Luego se transforma los datos según los índices generados. Notar que se utiliza `transform` para realizar esta transformación

In [ ]:
# Transformar los datos según los índices generados
df2 = modeloIndexador.transform(df)
df2.show()

Luego de esta transformación, se puede utilizar el índice `indiceSucursal` como entrada numérica para algún algoritmo de Machine Learning.

## 2.&nbsp;Atributos categóricos a numéricos usando One-hot Encoding

La idea de one-hot encoding es mapear cada categoría a un vector binario con un solo valor que indique la presencia de un atributo particular (una característica específica).

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [ ]:
# Data frame original
df.show()

### 2.1. Forma 1: Manipulación directa

En este caso, se puede manipular directamente el indexador y el conversor a one-hot encoding

In [ ]:
# Crear y aplicar un indexador
indexador = StringIndexer(inputCol="sucursal", outputCol="indiceSucursal")
# Transformar y aplicar el indexador al DataFrame
df2 = indexador.fit(df).transform(df)

# Definir One-Hot encoder
one_hot_encoder = OneHotEncoder(inputCol="indiceSucursal", outputCol="onehotSucursal")
# Transformar y aplicar OneHotEncoder al DataFrame
df3 = one_hot_encoder.fit(df2).transform(df2)
df3.show()

### Explicación de la Columna `onehotSucursal`

En la salida de la columna `onehotSucursal`, observamos la representación de los datos en formato **Sparse Vector** (vector disperso). Spark utiliza este formato para optimizar el almacenamiento de vectores con muchos ceros, como es el caso en las codificaciones One-Hot.

### Estructura de la Columna `onehotSucursal`

La salida tiene el siguiente formato: `(longitud del vector, [posiciones], [valores])`

1. **Longitud del vector:** Representa la cantidad total de elementos en el vector. En nuestro caso, el vector tiene 3 elementos.
2. **Posiciones:** Indica las posiciones en las que existen valores distintos de cero.
3. **Valores:** Lista los valores correspondientes a las posiciones indicadas, generalmente `1.0` en el caso de One-Hot Encoding.

### Ejemplos de Interpretación

- **(3, [0], [1.0])**:
  - Es un vector de longitud 3: `[1.0, 0.0, 0.0]`.
  - Esto significa que la sucursal tiene un índice de `0` (asignado a la sucursal "A").

- **(3, [2], [1.0])**:
  - Es un vector de longitud 3: `[0.0, 0.0, 1.0]`.
  - Esto significa que la sucursal tiene un índice de `2` (asignado a la sucursal "D").

- **(3, [], [])**:
  - Este caso indica un vector en el cual no se asignó ningún valor distinto de cero, común en ciertas salidas específicas de Spark.

### Importancia del Formato Sparse Vector

Este formato permite almacenar únicamente los valores necesarios, optimizando el espacio y mejorando el rendimiento en el procesamiento de datos con Spark. En otros entornos de programación, como pandas o scikit-learn, se suele utilizar la matriz completa, mostrando explícitamente todos los ceros y unos.


### 2.2. Forma 2: Usando un Pipeline


In [ ]:
from pyspark.ml import Pipeline

In [ ]:
# Indexador sin aplicarlo al DataFrame
string_indexer = StringIndexer(inputCol="sucursal", outputCol="indiceSucursal")
# OneHotEncoder sin aplicarlo al Dataframe
one_hot_encoder = OneHotEncoder(inputCol="indiceSucursal", outputCol="onehotSucursal")

# Pipeline con las etapas
pipeline = Pipeline(stages=[string_indexer,
                            one_hot_encoder])

# Obtener las asociaciones usando el pipeline
df2 = pipeline.fit(df)

# Transformar el DataFrame
df3 = df2.transform(df)

# Resultado
df3.show()

## 3.&nbsp;Generación de un vector columna (combinando otras columnas)

`VectorAssembler` combina un conjunto de columnas en un solo vector columna. Es útil para combinar atributos originales con aquellos generados por diferentes transformaciones aplicadas en PySpark. Esto es necesario para tener el formato que los modelos de ML de Spark utilizan.

`VectorAssembler` acepta los siguientes tipos de columnas: todos los tipos numéricos, tipos Booleanos, tipos vector. En cada fila, los valores de las columnas de entrada serán concatenados en un vector de un orden especificado.

Ejemplo: Si se tiene un DataFrame con las columnas id, campo1, campo2, campos3, y valor:

     id | campo1 | campo2 |   campos3   | valor
    ----|--------|--------|-------------|------
    204 |   18   |   1.0  | [3, 10, 20] |  5.9

donde `campos3` es una columna de vectores que contiene tres atributos. Se desea combinar `campo1`, `campo2` y `campos3` en un solo vector de atributos llamado `vatributos` para ser usado como predictor de `valor`. Si se indica que las columnas de entrada de `VectorAssembler` son `campo1`, `campo2` y `campos3`, y que la columna de salida es `valor`, luego de la transformación se obtendrá lo siguiente:

     id | campo1 | campo2 |   campos3   | valor |     vatributos
    ----|--------|--------|-------------|-------|----------------------
    204 |   18   |   1.0  | [3, 10, 20] |  5.9  | [18, 1.0, 3, 10, 20]

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors

In [ ]:
# Vector denso en Spark
vector = Vectors.dense([3, 10, 20])
vector

In [ ]:
# Creación de un data frame (de una fila)
df = spark.createDataFrame([(204, 18, 1.0, vector, 5.9),
                            (205, 25, 3.5, vector, 6.7)],
                           ["id", "campo1", "campo2", "campos3", "valor"])
df.show()

In [ ]:
# Objeto que juntará columnas para crear una sola columna
assembler = VectorAssembler(inputCols=["campo1", "campo2", "campos3"],
                            outputCol="vatributos")

# Transformar los datos según la columna creada
df2 = assembler.transform(df)
df2.show()

In [ ]:
df2.show(truncate=False)

In [ ]:
# Seleccionar solo las columnas vatributos y valor (usual como entrada a algoritmos supervizados)
df2.select("vatributos", "valor").show()